# Import

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [3]:
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

영어 번역 내용 제거

In [4]:
import os
import pandas as pd
import re

# 조건 설정 함수 정의
def set_conditions(df):
    condition_1 = df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))
    condition_2 = df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))
    condition_3 = df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
    condition_4 = df['키워드'].str.contains(r'\bBreak\b', na=False)
    return condition_1, condition_2, condition_3, condition_4

# 조건을 만족하는 행들로 새로운 데이터프레임 생성 함수 정의
def filter_dataframes(df, conditions):
    filtered_dfs = [df[condition].copy() for condition in conditions]
    return filtered_dfs

# 키워드 삭제 함수 정의
def remove_keywords(text, keywords):
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find(keywords[0])].strip()

# 한글 단어가 나오다가 영어 단어가 5번 이상 나오면 영어 단어가 나오기 시작한 단어부터 끝까지 삭제하는 함수 정의
def remove_english_after_five(text):
    english_words = re.findall(r'[a-zA-Z]+', text)
    if len(english_words) >= 5:
        fifth_english_word = english_words[4]
        start_index = text.find(fifth_english_word)
        return text[:start_index].strip()
    return text

# 키워드 수정 함수 정의
def apply_removals(filtered_dfs):
    filtered_dfs[0]['키워드'] = filtered_dfs[0]['키워드'].apply(lambda x: remove_keywords(x, ['영문', '번역', '오류', '전제']))
    filtered_dfs[1]['키워드'] = filtered_dfs[1]['키워드'].apply(lambda x: remove_keywords(x, ['구글', '번역기', '번역', '영문기사']))
    filtered_dfs[2]['키워드'] = filtered_dfs[2]['키워드'].apply(lambda x: remove_keywords(x, ['기사', '구글']))
    filtered_dfs[3]['키워드'] = filtered_dfs[3]['키워드'].apply(remove_english_after_five)
    return filtered_dfs

# 수정된 '키워드' 값을 원래 데이터프레임에 반영하는 함수 정의
def update_original_df(df, conditions, filtered_dfs):
    for condition, filtered_df in zip(conditions, filtered_dfs):
        df.loc[condition, '키워드'] = filtered_df['키워드']

# 메인 코드
conditions_train = set_conditions(train_df)
conditions_test = set_conditions(test_df)

filtered_train_dfs = filter_dataframes(train_df, conditions_train)
filtered_test_dfs = filter_dataframes(test_df, conditions_test)

filtered_train_dfs = apply_removals(filtered_train_dfs)
filtered_test_dfs = apply_removals(filtered_test_dfs)

update_original_df(train_df, conditions_train, filtered_train_dfs)
update_original_df(test_df, conditions_test, filtered_test_dfs)

In [5]:
# 수정된 데이터프레임을 CSV 파일로 저장
train_df.to_csv('train_df_translate_del.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('test_df_translate_del.csv', index=False, encoding='utf-8-sig')

.